In [ ]:
Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг
Результат классификации запишите в столбец class

In [3]:
import pandas as pd
data = pd.read_csv('ratings.csv','r', delimiter=',')

def classify_rating (row):
    if row.rating>=4.5:
        i = 'высокий рейтинг'
    elif 2<row.rating<4.5:
        i = 'средний рейтинг'
    elif row.rating<=2:
        i = 'низкий рейтинг'
    return (i)
        
data['class']=data.apply(classify_rating, axis=1)
data.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


In [ ]:
Задание 2
Необходимо написать гео-классификатор, который каждой строке сможет выставить 
географическую принадлежность определенному региону. 
Т. е. если поисковый запрос содержит название города региона, то в столбце 
‘region’ пишется название этого региона. Если поисковый запрос не содержит 
названия города, то ставим ‘undefined’.
Результат классификации запишите в отдельный столбец region.

In [112]:
geo_data = { 
    'Центр': ['москва', 'оренбург', 'воронеж', 'нижний новгород', 'казань'], 
    'Северо-Запад': ['петербург', 'псков', 'мурманск'], 
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск', 'тюмень']
}

sourse = pd.read_html('https://visasam.ru/russia/goroda/uroven-zhizni-v-rossii.html',header = 0)[0] 
cities = sourse.drop(city.columns[[1]], axis=1)
cities.columns=['city']
cities['city_lower']=cities.city.str.lower()

def classify_city (row):
    z = 0
    for key, value in geo_data.items():
        for i in value:
            if i in  row[1]:
                z+=1
                return key
    if z==0:
        return 'undefined'
                
      
cities['region']=cities.apply(classify_city, axis=1)
cities.drop(cities.columns[[1]], axis=1).head(10)

,city,region
0,Казань,Центр
1,Тюмень,Дальний Восток
2,Набережные Челны,undefined
3,Грозный,undefined
4,Томск,undefined
5,Оренбург,Центр
6,Санкт-Петербург,Северо-Запад
7,Пенза,undefined
8,Воронеж,Центр
9,Кемерово,undefined


In [ ]:
Задание 3 
Проверить гипотезу: с ростом года выпуска фильма его средний рейтинг становится ниже.
(на материалах прошлого домашнего задания)
Алгоритм:
В переменную years запишите список из всех годов с 1950 по 2010.
Напишите функцию production_year, которая каждой строке из названия фильма 
выставляет год выпуска. 
Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому 
используйте следующий алгоритм: 
Для каждой строки пройдите по всем годам списка years
        если номер года присутствует в названии фильма, то функция возвращает 
        этот год как год выпуска
        если ни один из номеров года списка years не встретился в названии 
        фильма, то возвращается 1900 год
Запишите год выпуска фильма в новый столбец ‘year’
Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и 
отсортируйте результат по убыванию рейтинга

In [247]:
import zipfile as zf
import requests, io, re, numpy, datetime
from statistics import mean

r = requests.get('http://files.grouplens.org/datasets/movielens/ml-100k.zip')
z = zf.ZipFile(io.BytesIO(r.content))
rate  = pd.read_table(z.extract('ml-100k/u.data'), names=['user_id', 'item_id', 'rating', 'timestamp'], header =0)
films = pd.read_table(z.extract('ml-100k/u.item'), encoding = "ISO-8859-1", delimiter='|', header = None)
f2 = films.drop(films.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]], axis=1)
f2.columns=['item_id', 'film']
print(rate.head(2)) 
# Дубль прошлого домашнего задания, скачиваем архив и удаляем все лишнее

   user_id  item_id  rating  timestamp
0      186      302       3  891717742
1       22      377       1  878887116


In [280]:
def set_year(row):
    z = re.findall('(\d\d\d\d)', row.film)
    if len(z) == 0 or z == 0 or z == None:
        z = 9999
    else:
        z = int(z[0])
    return z    

f2['year_of_film']= f2.apply(set_year, axis=1)
f2[ f2.year_of_film != 9999 ]
print(f2)
# выделяем год выпуска фильма в отдельный столбец + выбрасываем совсем плохие строки из выборки

      item_id                                       film  year_of_film
0           1                           Toy Story (1995)          1995
1           2                           GoldenEye (1995)          1995
2           3                          Four Rooms (1995)          1995
3           4                          Get Shorty (1995)          1995
4           5                             Copycat (1995)          1995
...       ...                                        ...           ...
1677     1678                          Mat' i syn (1997)          1997
1678     1679                           B. Monkey (1998)          1998
1679     1680                       Sliding Doors (1998)          1998
1680     1681                        You So Crazy (1994)          1994
1681     1682  Scream of Stone (Schrei aus Stein) (1991)          1991

[1682 rows x 3 columns]


In [267]:
f3=rate.merge(f2, how = 'right', left_on='item_id', right_on= 'item_id')
print(f3.head(2))
# join таблиц по коду фильма - item_id

   user_id  item_id  rating  timestamp                      film  year_of_film
0      186      302       3  891717742  L.A. Confidential (1997)          1997
1      191      302       4  891560253  L.A. Confidential (1997)          1997


In [ ]:
def set_time(row):
    return datetime.datetime.fromtimestamp(f3.timestamp[0]).year

def period(row):
    per = row['year_of_rate'] - row['year_of_film']
    return per

f3['year_of_rate']= f3.apply(set_time, axis=1)
f3['period_of_rate']= f3.apply(period, axis=1)
f3 = f3.drop(f3.columns[[0,1,3,4]], axis=1)
print(f3)
# добавляем столбцы : год выставления оценки (по timestamp) 
# и период (в годах  с даты выхода фильма до даты выставления оценки)

In [273]:
yof2 = pd.cut(f3['year_of_film'], [1900, 1950, 1960,1970, 1980, 1990, 1995, 2000, 2005, 2010]) 
yor2 = pd.cut(f3['period_of_rate'], [-999, 0, 1, 2, 3, 5, 999])
#table = pd.pivot_table(f3, values =['rating'], index = yof2, columns = ['year_of_rate'], aggfunc = mean) 
table = pd.pivot_table(f3, values =['rating'], index = yof2, columns = yor2, aggfunc = 'mean') 
print (table)

# сводная таблица : год выхода фильма / "возраст рейтинга"(лет с даты фильма до даты timestamp) |
# среднее значение рейтинга  (было желание выйти на винтажный анализ - но данных не хватило)

                  rating                                                
period_of_rate (-999, 0]    (0, 1]    (1, 2]   (2, 3]   (3, 5]  (5, 999]
year_of_film                                                            
(1900, 1950]         NaN       NaN       NaN      NaN      NaN  3.954029
(1950, 1960]         NaN       NaN       NaN      NaN      NaN  3.950700
(1960, 1970]         NaN       NaN       NaN      NaN      NaN  3.862788
(1970, 1980]         NaN       NaN       NaN      NaN      NaN  3.875665
(1980, 1990]         NaN       NaN       NaN      NaN      NaN  3.729293
(1990, 1995]         NaN       NaN       NaN  3.41304  3.46482  3.597387
(1995, 2000]    3.297297  3.315706  3.310358      NaN      NaN       NaN
(2000, 2005]    3.969112       NaN       NaN      NaN      NaN       NaN


In [221]:
f3[ f3.period_of_rate < 0 ]
# нашел очевидную ошибку в кодировке или трактовке timestamp

,rating,year_of_film,year_of_rate,period_of_rate
19641,3,2001,1998,-3
19642,4,2001,1998,-3
19643,3,2001,1998,-3
19644,5,2001,1998,-3
19645,4,2001,1998,-3
...,...,...,...,...
93973,4,2000,1998,-2
93974,3,2000,1998,-2
93975,5,2000,1998,-2
93976,3,2000,1998,-2
